## Sample Data Load (SQLite)

In [1]:
import json

schema_file_path = './db_schema.json'
with open(schema_file_path, 'r') as file:
    schema_data = json.load(file)

### Table Creation

In [2]:
import sqlite3
import csv
import pandas as pd

db_name = 'sample.db'
table_name = 'food_sales_table'
sample_data_path = 'demo_data.csv'
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [3]:
drop_table_query = f'''
DROP TABLE IF EXISTS {table_name}
'''

create_table_query = f'''
CREATE TABLE {table_name} (
'''

for column in schema_data[table_name]['cols']:
    column_name = column['col']
    data_type = column['type']
    create_table_query += f"    {column_name} {data_type},\n"

create_table_query = create_table_query.rstrip(',\n') + '\n)'
print(create_table_query)


CREATE TABLE food_sales_table (
    reference_month varchar,
    headquaters_current_id varchar,
    headquaters_current_name varchar,
    business_current_department_code varchar,
    business_current_department_name varchar,
    sales_group_current_code varchar,
    sales_group_current_name varchar,
    route_current_business_department_code varchar,
    route_current_business_department_name varchar,
    manager_id varchar,
    manager_name varchar,
    head_office_code varchar,
    head_office_name varchar,
    customer_id varchar,
    customer_name varchar,
    product_number varchar,
    product_name varchar,
    large_product_category varchar,
    large_product_category_name varchar,
    middle_product_category varchar,
    middle_product_category_name varchar,
    small_product_category varchar,
    small_product_category_name varchar,
    detailed_product_category varchar,
    detailed_product_category_name varchar,
    pb_div_cd varchar,
    pb_div_nm varchar,
    sales_valu

In [4]:
try:
    cursor.execute(drop_table_query)
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Table {table_name} has been recreated successfully.")
    
    with open(sample_data_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.reader(csv_file)
        headers = next(csv_reader)

        placeholders = ','.join(['?' for _ in headers])
        insert_query = f'''
        INSERT INTO {table_name} ({','.join(headers)})
        VALUES ({placeholders})
        '''

        for row in csv_reader:
            cursor.execute(insert_query, row)

    conn.commit()
    print("Data has been inserted successfully.")

except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    conn.close()

Table food_sales_table has been recreated successfully.
Data has been inserted successfully.


### Test Run

In [7]:
import pandas as pd

conn = sqlite3.connect(db_name)

with open('sample_queries.json', 'r') as f:
    queries = json.load(f)

for n, query_info in enumerate(queries): 
    query = query_info['sql_query']
    question = query_info['question']
    df = pd.read_sql_query(query, conn)
    print(f"Question {n}:", question)
    print(f"Result {n}:", df)

Question 0: 당월 매출 조회
Result 0:   reference_month  current_month_sales
0         2024.11         2.121339e+07
Question 1: 전월비 매출 조회
Result 1:    current_month_sales  previous_month_sales  mom_change
0         2.121339e+07          2.249515e+07   -5.697934
Question 2: 최근 3개월 사업부 매출
Result 2:    business_current_department_code business_current_department_name  \
0                             18075                            사업부_2   
1                             18075                            사업부_2   
2                             18075                            사업부_2   
3                             66484                            사업부_3   
4                             66484                            사업부_3   
5                             66484                            사업부_3   
6                             80841                            사업부_1   
7                             80841                            사업부_1   
8                             80841                          

### OpenSearch Indexing

In [28]:
#!pip install opensearch-py

In [37]:
import os
from dotenv import load_dotenv
from opensearchpy import OpenSearch, RequestsHttpConnection

load_dotenv()

host = os.getenv('OPENSEARCH_HOST')
user = os.getenv('OPENSEARCH_USER')
password = os.getenv('OPENSEARCH_PASSWORD')

os_client = OpenSearch(
    hosts = [{'host': host.replace("https://", ""), 'port': 443}],
    http_auth = (user, password),
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [38]:
mapping = {
    "settings": {
        "index.knn": True,
        "index.knn.algo_param.ef_search": 512
    },
    "mappings": {
        "properties": {
            "question": {
                "type": "text",
                "analyzer": "standard"
            },
            "sql_query": {
                "type": "text"
            },            
            "question_embedding": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "engine": "faiss",
                    "name": "hnsw",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    },
                    "space_type": "l2"
                }
            }
        }
    }
}

In [ ]:
index_name = "sample_queries"

def create_index(index_name):
    if os_client.indices.exists(index_name):
        os_client.indices.delete(index_name)
        print(f"Existing index '{index_name}' deleted.")

    os_client.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created successfully.")

create_index(index_name)

In [43]:
import boto3

bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")
embed_model = "amazon.titan-embed-text-v2:0"

with open(f"sample_queries.json", 'r', encoding='utf-8') as f:
    documents = json.load(f)

for doc in documents:
    response = bedrock_client.invoke_model(
                modelId=embed_model,
                body=json.dumps({"inputText": doc['sql_query']})
            )
    doc['question_embedding'] = json.loads(response['body'].read())['embedding']
    os_client.index(
        index=index_name,
        body=doc
    )
